In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
academic_level_data=pd.read_csv("student-mat.csv", delimiter=";")
academic_level_data.info()# 382 students


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      395 non-null    object
 1   sex         395 non-null    object
 2   age         395 non-null    int64 
 3   address     395 non-null    object
 4   famsize     395 non-null    object
 5   Pstatus     395 non-null    object
 6   Medu        395 non-null    int64 
 7   Fedu        395 non-null    int64 
 8   Mjob        395 non-null    object
 9   Fjob        395 non-null    object
 10  reason      395 non-null    object
 11  guardian    395 non-null    object
 12  traveltime  395 non-null    int64 
 13  studytime   395 non-null    int64 
 14  failures    395 non-null    int64 
 15  schoolsup   395 non-null    object
 16  famsup      395 non-null    object
 17  paid        395 non-null    object
 18  activities  395 non-null    object
 19  nursery     395 non-null    object
 20  higher    

In [78]:
academic_level_data

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


In [79]:
academic_level_data.shape

(395, 33)

In [80]:
from enum import Enum
class ColumnType(Enum):
    ABSOLUTE = 1
    ORDER = 2
    NAME = 3

In [81]:
categorical_columns = academic_level_data.columns[academic_level_data.dtypes == 'object']
colunm_type_map = dict()
for column in categorical_columns:
    colunm_type_map[column] = ColumnType.NAME

colunm_type_map['age'] = ColumnType.ABSOLUTE
colunm_type_map['falures'] = ColumnType.ABSOLUTE
colunm_type_map['absences'] = ColumnType.ABSOLUTE

for column_name in academic_level_data:
    if colunm_type_map.get(column_name) is None:
        colunm_type_map[column_name] = ColumnType.ORDER
colunm_type_map

{'school': <ColumnType.NAME: 3>,
 'sex': <ColumnType.NAME: 3>,
 'address': <ColumnType.NAME: 3>,
 'famsize': <ColumnType.NAME: 3>,
 'Pstatus': <ColumnType.NAME: 3>,
 'Mjob': <ColumnType.NAME: 3>,
 'Fjob': <ColumnType.NAME: 3>,
 'reason': <ColumnType.NAME: 3>,
 'guardian': <ColumnType.NAME: 3>,
 'schoolsup': <ColumnType.NAME: 3>,
 'famsup': <ColumnType.NAME: 3>,
 'paid': <ColumnType.NAME: 3>,
 'activities': <ColumnType.NAME: 3>,
 'nursery': <ColumnType.NAME: 3>,
 'higher': <ColumnType.NAME: 3>,
 'internet': <ColumnType.NAME: 3>,
 'romantic': <ColumnType.NAME: 3>,
 'age': <ColumnType.ABSOLUTE: 1>,
 'absences': <ColumnType.ABSOLUTE: 1>,
 'Medu': <ColumnType.ORDER: 2>,
 'Fedu': <ColumnType.ORDER: 2>,
 'traveltime': <ColumnType.ORDER: 2>,
 'studytime': <ColumnType.ORDER: 2>,
 'failures': <ColumnType.ORDER: 2>,
 'famrel': <ColumnType.ORDER: 2>,
 'freetime': <ColumnType.ORDER: 2>,
 'goout': <ColumnType.ORDER: 2>,
 'Dalc': <ColumnType.ORDER: 2>,
 'Walc': <ColumnType.ORDER: 2>,
 'health': <Colu

In [82]:
def calcDiffMatrix(data, column_type_map):
    matrix = np.zeros((data.shape[0], data.shape[0]))
    print("here")
    for i, row1 in data[:-1].iterrows():
        for j, row2 in data[i+1:].iterrows():
            if j != i:
                matrix[j, i] = matrix[i, j] = calcObjectsDiff(row1, row2, data, column_type_map)
            print(i, ' diff beetwen obj_i = ', i, 'and obj_j = ', j, ' = ', matrix[i,j])
    return matrix
            
            
def calcObjectsDiff(obj1, obj2, data, column_type_map):
    diff = 0
    for i, column_name in enumerate(data):
        if column_type_map[column_name] == ColumnType.NAME:
            diff += calcNameColumnDiff(obj1[i], obj2[i], data[column_name]) ** 2
        elif column_type_map[column_name] == ColumnType.ORDER:
            diff += calcOrderColumnDiff(obj1[i], obj2[i], data[column_name]) ** 2
        elif column_type_map[column_name] == ColumnType.ABSOLUTE:
            diff += calcAbsoluteColumnDiff(obj1[i], obj2[i], data[column_name]) ** 2
    return np.sqrt(diff)/np.sqrt(len(data))

def calcNameColumnDiff(obj1_data, obj2_data, column_data):
    return (1/len(column_data)) * calcNotNormolizedNameDiff(obj1_data, obj2_data, column_data)

def calcNotNormolizedNameDiff(obj1_data, obj2_data, column_data):
    diff = 0
    for value in column_data:
        if obj1_data != value == obj2_data or obj1_data == value != obj2_data:
            diff += 1
    return diff

def calcOrderColumnDiff(obj1_data, obj2_data, column_data):
    diff = 0
    for value in column_data:
        if obj1_data > value > obj2_data or obj1_data < value < obj2_data:
            diff += 1
        elif obj1_data == value != obj2_data or obj1_data != value == obj2_data:
            diff += 0.5
    return diff/len(column_data)

def calcAbsoluteColumnDiff(obj1_data, obj2_data, column_data):
    return np.abs(obj1_data - obj2_data) / (np.max(column_data) - np.min(column_data))

In [ ]:
diff_matrix = calcDiffMatrix(academic_level_data, colunm_type_map)
diff_matrix

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44


In [ ]:
sub_matrix = diff_matrix[diff_matrix > 0.2]
sub_matrix